# LangChain 实战：电视和冰箱销售聊天机器人

## 使用 GPT-3.5 构造销售话术数据

使用 ChatGPT 构造销售数据的 Prompt 示例：

```

你是中国顶级的液晶电视机和冰箱销售，现在培训职场新人，请给出100条实用的销售话术。

每条销售话术以如下格式给出：
[客户问题]
[销售回答]


```

chatgpt-3.5 回复结果：

```
护眼功能：

[客户问题]：这款电视有护眼功能吗？
[销售回答]：是的，这款电视配备了护眼功能，它可以通过减少蓝光辐射来减轻眼睛疲劳，特别适合长时间观看。

[客户问题]：如何开启或调整护眼模式？
[销售回答]：您可以在设置中轻松开启或调整护眼模式，以适应您的视觉需求。

[客户问题]：护眼模式会影响画质吗？
[销售回答]：不会的，护眼模式只会减少蓝光辐射，不影响画质，您仍然可以享受高清画面。

[客户问题]：这款电视的护眼功能是否可自动调整？
[销售回答]：是的，它可以自动检测光线情况，并自动调整以最大程度减少眼睛的不适感。

回放功能：

[客户问题]：这款电视是否支持节目回放？
[销售回答]：当然，它支持节目回放功能，您可以随时回看您喜欢的节目。

[客户问题]：如何使用节目回放功能？
[销售回答]：您只需打开回放菜单，选择您想要观看的节目，然后按播放按钮即可。

[客户问题]：有多长时间的节目回放记录？
[销售回答]：这取决于电视型号，通常有 [节目回放时间] 的记录，足够您回看错过的节目。

[客户问题]：能否暂停和快进回放的节目？
[销售回答]：是的，您可以随时暂停、快进或倒退回放的节目，以满足您的观看需求。

[客户问题]：是否可以在回放期间录制节目？
[销售回答]：某些型号的电视支持在回放期间录制节目，这样您可以随时保存您喜欢的内容。
```


## 使用 Document Transformers 模块来处理原始数据


将 ChatGPT 生成的结果保存到 [electron_sales_data.txt](electron_sales_data.txt) 文件中

In [1]:
with open("electron_sales_data.txt", encoding="utf-8") as f:
    electron_sales = f.read()

### 使用 CharacterTextSplitter 来进行文本分割

- 基于单字符来进行文本分割（separator）
- 基于字符数来决定文本块长度（chunk_size）

参考示例：

```python
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(        
    separator = "\n\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
    is_separator_regex = False,
)
```


In [2]:
from langchain.text_splitter import CharacterTextSplitter

In [3]:
text_splitter = CharacterTextSplitter(        
    separator = r'\d+\.',
    chunk_size = 100,
    chunk_overlap  = 0,
    length_function = len,
    is_separator_regex = True,
)

In [4]:
docs = text_splitter.create_documents([electron_sales])

In [5]:
docs[0]

Document(page_content='[客户问题]：这款电视有多大的屏幕？\n[销售回答]：这款电视有 [屏幕尺寸] 寸的屏幕，提供清晰的高清图像。', metadata={})

In [6]:
len(docs)

104

### 使用 Faiss 作为向量数据库，持久化存储房产销售 问答对（QA-Pair）

In [7]:
# get a token: https://platform.openai.com/account/api-keys

from getpass import getpass

OPENAI_API_KEY = getpass()

 ········


In [8]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS

db = FAISS.from_documents(docs, OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY))

In [9]:
query = "介绍液晶电视的尺寸"

In [10]:
answer_list = db.similarity_search(query)

In [11]:
for ans in answer_list:
    print(ans.page_content + "\n")

[客户问题]：这款电视有多大的屏幕？
[销售回答]：这款电视有 [屏幕尺寸] 寸的屏幕，提供清晰的高清图像。

[客户问题]：这款电视有多薄？
[销售回答]：这款电视非常薄，采用了先进的薄型面板技术，节省空间。

[客户问题]：这款电视支持哪种高动态范围（HDR）？
[销售回答]：这款电视支持 [HDR标准]，使画面更明亮、更锐利。

[客户问题]：电视的刷新率是多少？
[销售回答]：这款电视的刷新率为 [刷新率] Hz，确保动态画面流畅。



In [12]:
db.save_local("electron_sales")

### 使用 retriever 从向量数据库中获取结果

#### 使用参数 `k` 指定返回结果数量


In [13]:
topK_retriever = db.as_retriever(search_kwargs={"k": 3})

In [14]:
topK_retriever

VectorStoreRetriever(tags=['FAISS'], metadata=None, vectorstore=<langchain.vectorstores.faiss.FAISS object at 0x00000191971CE5D0>, search_type='similarity', search_kwargs={'k': 3})

In [15]:
docs = topK_retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[客户问题]：这款电视有多大的屏幕？
[销售回答]：这款电视有 [屏幕尺寸] 寸的屏幕，提供清晰的高清图像。

[客户问题]：这款电视有多薄？
[销售回答]：这款电视非常薄，采用了先进的薄型面板技术，节省空间。

[客户问题]：这款电视支持哪种高动态范围（HDR）？
[销售回答]：这款电视支持 [HDR标准]，使画面更明亮、更锐利。



In [16]:
docs = topK_retriever.get_relevant_documents("你们有没有70寸的液晶电视和节能1级的冰箱？")

In [17]:
for doc in docs:
    print(doc.page_content + "\n")

[客户问题]：冰箱是否具有节能特性？
[销售回答]：我们的冰箱拥有高效的节能特性，能够帮助您降低用电成本。

[客户问题]：冰箱的能效等级是什么？
[销售回答]：这款冰箱拥有 [能效等级]，节约能源并降低用电成本。

[客户问题]：购买这款电视/冰箱是否有机会赢得抽奖？
[销售回答]：是的，购买这款产品，您将有机会参加抽奖，赢得丰厚奖品。



#### 使用 similarity_score_threshold 设置阈值，提升结果的相关性质量

In [18]:
retriever = db.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.8}
)

In [19]:
docs = retriever.get_relevant_documents(query)
for doc in docs:
    print(doc.page_content + "\n")

[客户问题]：这款电视有多大的屏幕？
[销售回答]：这款电视有 [屏幕尺寸] 寸的屏幕，提供清晰的高清图像。



### 提取向量数据库中的`销售回答`

In [20]:
docs = retriever.get_relevant_documents(query)

In [21]:
docs[0].page_content

'[客户问题]：这款电视有多大的屏幕？\n[销售回答]：这款电视有 [屏幕尺寸] 寸的屏幕，提供清晰的高清图像。'

In [31]:
docs[0].page_content.split("[销售回答]：")

['[客户问题]：这款电视有多大的屏幕？\n', '这款电视有 [屏幕尺寸] 寸的屏幕，提供清晰的高清图像。']

In [32]:
ans = docs[0].page_content.split("[销售回答]：")[-1]

In [33]:
ans

'这款电视有 [屏幕尺寸] 寸的屏幕，提供清晰的高清图像。'

#### 尝试各种问题

In [34]:
from typing import List

def sales(query: str, score_threshold: float=0.8) -> List[str]:
    retriever = db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"score_threshold": score_threshold})    
    docs = retriever.get_relevant_documents(query)
    ans_list = [doc.page_content.split("[销售回答]：")[-1] for doc in docs]

    return ans_list

In [35]:
query = "我想要有护眼功能和节目回放功能的电视"

print(sales(query))

['是的，这款电视配备了护眼功能，它可以通过减少蓝光辐射来减轻眼睛疲劳，特别适合长时间观看。', '当然，它支持节目回放功能，您可以随时回看您喜欢的节目。']


In [36]:
print(sales(query, 0.75))

['是的，这款电视配备了护眼功能，它可以通过减少蓝光辐射来减轻眼睛疲劳，特别适合长时间观看。', '当然，它支持节目回放功能，您可以随时回看您喜欢的节目。', '是的，它可以自动检测光线情况，并自动调整以最大程度减少眼睛的不适感。', '这款电视配备了智能功能，包括 [列出智能功能]，使您的娱乐体验更丰富。']


In [37]:
query = "价格3000以内"

print(f"score:0.8 ans: {sales(query)}\n")
print(f"score:0.75 ans: {sales(query, 0.75)}\n")
print(f"score:0.5 ans: {sales(query, 0.5)}\n")

C:\Users\yanca\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain\vectorstores\base.py:266: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.8
  warnings.warn(


score:0.8 ans: []



C:\Users\yanca\AppData\Local\Programs\Python\Python311\Lib\site-packages\langchain\vectorstores\base.py:266: UserWarning: No relevant docs were retrieved using the relevance score threshold 0.75
  warnings.warn(


score:0.75 ans: []

score:0.5 ans: ['是的，我们提供价格匹配保证，确保您获得最佳价格。', '是的，我们提供价格保护政策，如果您在购买后发现更低价格，我们会退还差价。', '购买这款产品，您可以获得 [金额] 的优惠券，用于以后的购物。', '是的，我们提供 0% 利率分期付款选项，让您更轻松地购买。']



#### 当向量数据库中没有合适答案时，使用大语言模型能力

In [38]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, openai_api_key=OPENAI_API_KEY)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [39]:
qa_chain({"query": query})

{'query': '价格3000以内', 'result': '请问您是在询问什么产品或服务的价格在3000以内呢？'}

In [40]:
qa_chain({"query": "电视加上冰箱是否更加的优惠"})

{'query': '电视加上冰箱是否更加的优惠', 'result': '很抱歉，我无法回答您的问题。'}

In [41]:
print(sales("电视加上冰箱是否更加的优惠"))

['购买这款产品，您可以获得 [金额] 的优惠券，用于以后的购物。', '是的，购买这款产品，您将有机会参加限量特别活动，获得独特的体验。', '是的，购买这款产品，您将有机会参加抽奖，赢得丰厚奖品。', '购买指定型号的电视/冰箱，您可以获得免费安装服务。']


## 加载 FAISS 向量数据库已有结果

In [42]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS

db = FAISS.load_local("electron_sales", OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY))

In [43]:
from langchain.chains import RetrievalQA
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, openai_api_key=OPENAI_API_KEY)
qa_chain = RetrievalQA.from_chain_type(llm,
                                       retriever=db.as_retriever(search_type="similarity_score_threshold",
                                                                 search_kwargs={"score_threshold": 0.8}))

In [44]:
qa_chain({"query": "我想买空调，你们有么"})

{'query': '我想买空调，你们有么',
 'result': '很抱歉，我是一个语言模型，无法提供实际产品。建议您去当地的家电商店或者在线购物平台上查找空调产品。'}

In [45]:
# 输出内部 Chain 的日志
qa_chain.combine_documents_chain.verbose = True

In [46]:
qa_chain({"query": "我想买空调，你们有么"})



> Entering new StuffDocumentsChain chain...

> Finished chain.


{'query': '我想买空调，你们有么',
 'result': '很抱歉，我是一个语言模型，无法提供实际产品。建议您去当地的家电商店或者在线购物平台上查找空调产品。'}

In [47]:
# 返回向量数据库的检索结果
qa_chain.return_source_documents = True

In [48]:
result = qa_chain({"query": "我想买空调，你们有么"})



> Entering new StuffDocumentsChain chain...

> Finished chain.


In [49]:
result

{'query': '我想买空调，你们有么',
 'result': '很抱歉，我是一个语言模型，无法提供实际产品。建议您去当地的家电商店或者在线购物平台上查找空调产品。',
 'source_documents': []}